## __Tarea 1 - Parcial 1 "Distancias"__
### _Pasos para resolver la tarea_
- Abrir MySQL para ejecutar los scripts de SQL y crear las tablas con sus respectivos registros para posteriormente exportar estos datos en un archivo CSV.

- Abrir los archivos CSV en Python para analizar y limpiar los datos que sean inconsistentes, columnas innecesarias y duplicados.

- Unir los dataframes limpios por el código postal y aplicar la fórmula de Haservine para obtener la distancia entre ambos lugares.

- Filtrar por las distancias mayores a 10 Km.

- Inferencia y análisis de los datos obtenidos.


In [49]:
import numpy as np
import pandas as pd

In [50]:
def haversine(lon1, lat1, lon2, lat2):
    #Convertir grados decimales a radianes
    lon1 = np.radians(lon1.astype(float))
    lat1 = np.radians(lat1.astype(float))
    lon2 = np.radians(lon2.astype(float))
    lat2 = np.radians(lat2.astype(float))

    #Formula de Haversine
    dis_lon = lon2 - lon1
    dis_lat  = lat2 - lat1
    a = np.sin(dis_lat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dis_lon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    #Radio de la tierra en KM
    r = 6371 

    return c * r

In [51]:
#Cargar los datos del archivo CSV en un dataframe
df_colonias = pd.read_csv('cat_colonias.csv')

#Verificar las filas donde la latitud es inválida (< -90 o > 90)
latitud_invalida_col = df_colonias[
    (df_colonias['lat'] < -90) | (df_colonias['lat'] > 90)
]

#Verificar las filas donde la longitud es inválida (< -180 o > 180)
longitud_invalida_col = df_colonias[
    (df_colonias['lon'] < -180) | (df_colonias['lon'] > 180)
]

#En caso de que hubiera datos inválidos para la latitud y la longitud se deben eliminar.

#Eliminar las filas que contengan datos nulos
df_limpio_colonias = df_colonias.dropna()


In [54]:
#Cargar los datos del archivo CSV en un dataframe
df_edificios = pd.read_csv('edificios_escolares.csv')

# Se identificó que las columnas de latitud y longitud contienen valores String, por lo que se convertirán a numéricos (double)
df_edificios['latitud'] = pd.to_numeric(df_edificios['latitud'], errors='coerce')
df_edificios['longitud'] = pd.to_numeric(df_edificios['longitud'], errors='coerce')

#Verificar las filas donde la latitud es inválida (< -90 o > 90)
latitud_invalida_ed = df_edificios[
    (df_edificios['latitud'] < -90) | (df_edificios['latitud'] > 90)
]

#Verificar las filas donde la longitud es inválida (< -180 o > 180)
longitud_invalida_ed = df_edificios[
    (df_edificios['longitud'] < -180) | (df_edificios['longitud'] > 180)
]

#Existen datos inválidos para la longitud, que hay que eliminar.
df_limpio_edificios = df_edificios[
    (df_edificios['longitud'] >= -180) & 
    (df_edificios['longitud'] <= 180)
]

#Existen datos nulos en latitud que hay que eliminar.
df_limpio_edificios = df_limpio_edificios.dropna(subset=['latitud'])

#Eliminar columnas que no se usaran para el análisis
df_limpio_edificios = df_limpio_edificios.drop(columns=[
    'desc_tipo_centro', 'desc_turno', 'desc_servicio', 
    'desc_sostenimiento', 'desc_nivel_educativo', 
    'desc_subnivel_educativo', 'desc_status', 'desc_modalidad',
    'desc_categoria_poblacion','fecha_alta', 'fecha_baja', 'fecha_clausura', 
    'fecha_reapertura', 'fecha_cambio','tipo_centro', 'turno', 'servicio', 'sostenimiento',
    'nivel_educativo', 'subnivel_educativo', 'status',
    'modalidad', 'categoria_poblacion','region', 'des_region', 'nombre_de_municipio',
    'nombre_de_localidad', 'cve_tipo_asentamiento',
    'tipo_asentamiento', 'cve_asentamiento','numero_exterior', 'numero_interior'
])

#Eliminar filas que su código postal sea 0 o null
df_limpio_edificios = df_limpio_edificios[df_limpio_edificios['codigo_postal'] != 0]

In [57]:
#Validar que los dataframes estén Limpios.
print("=== VERIFICACIÓN DE DATAFRAMES LIMPIOS ===")
print(f"Colonias limpias: {df_limpio_colonias.shape}")
print(f"Edificios limpios: {df_limpio_edificios.shape}")

print("\n=== VALORES NULOS EN COORDENADAS ===")
print("Colonias - lat nulos:", df_limpio_colonias['lat'].isnull().sum())
print("Colonias - lon nulos:", df_limpio_colonias['lon'].isnull().sum())
print("Edificios - latitud nulos:", df_limpio_edificios['latitud'].isnull().sum())
print("Edificios - longitud nulos:", df_limpio_edificios['longitud'].isnull().sum())

=== VERIFICACIÓN DE DATAFRAMES LIMPIOS ===
Colonias limpias: (2692, 12)
Edificios limpios: (8822, 8)

=== VALORES NULOS EN COORDENADAS ===
Colonias - lat nulos: 0
Colonias - lon nulos: 0
Edificios - latitud nulos: 0
Edificios - longitud nulos: 0


In [ ]:
#renombrar columna de codigo postal (cve_codpost) el el df_limpio_colonias como en el df_limpio_edificios para poder hacer un merge
df_limpio_colonias = df_limpio_colonias.rename(columns={'cve_codpost' : 'codigo_postal'})

#Unir los dataframes para calcular las distancias.
df_merged = pd.merge(df_limpio_edificios, df_limpio_colonias, on='codigo_postal', how='inner', suffixes=('_escuela', '_colonia'))

In [68]:
#crear una nueva columna en el df_merged que contenga la distancia entre la escuela y la colonia usando la función Haservine
df_merged['distancia_km'] = haversine(
    df_merged['latitud'], df_merged['longitud'], df_merged['lat'], df_merged['lon']
)

In [81]:
# Filter the DataFrame for schools with a distance of 10 km or more
df_final = df_merged[df_merged['distancia_km'] >= 10]
df_final = df_final.drop_duplicates()

# Display the final results, showing key information
print("Edificios escolares a 10 km o más de distancia de su código postal:")
df_final[['nombre', 'codigo_postal', 'asentamiento', 'nombre_colonia', 'distancia_km']]

Edificios escolares a 10 km o más de distancia de su código postal:


,nombre,codigo_postal,asentamiento,nombre_colonia,distancia_km
1844,SUPERVISION DE EDUCACION SECUNDARIA GENERAL NU...,25000,SALTILLO ZONA CENTRO,Saltillo Zona Centro,271.780826
1860,SUPERVISION DE ZONA ESCOLAR DE EDUCACION PRIMA...,25000,SALTILLO ZONA CENTRO,Saltillo Zona Centro,11.297074
1914,JEFATURA DE SECTOR DE EDUCACION PREESCOLAR NUM...,25000,SALTILLO ZONA CENTRO,Saltillo Zona Centro,56.522008
1921,COLEGIO DE EDUCACION PROFESIONAL TECNICA DEL E...,25204,LOS PINOS,San Patricio Plus,277.747444
1922,COLEGIO DE EDUCACION PROFESIONAL TECNICA DEL E...,25204,LOS PINOS,San José,277.747444
...,...,...,...,...,...
53172,CENTRO DE RECURSOS DE INFORMACION E INTEGRACIO...,25010,LA FUENTE 2A AMPLIACIÓN,Las Coquetas,258.129003
53173,CENTRO DE RECURSOS DE INFORMACION E INTEGRACIO...,25010,LA FUENTE 2A AMPLIACIÓN,Vista Hermosa,258.129003
53174,CENTRO DE RECURSOS DE INFORMACION E INTEGRACIO...,25010,LA FUENTE 2A AMPLIACIÓN,Privada de San Javier,258.129003
53237,INSTITUTO LAGUNERO DE AUDICION Y LENGUAJE,25045,PORTAL DE SANTIAGO,Zapateros,268.783658


## __INFERENCIA:__
Existe una inconsistencia en la ubicación de algunos edificios escolares o colonias, donde las coordenadas registradas no corresponden a la ubicación real dentro del código postal declarado. Esto se puede inferir debido a las grandes distacias entre las colonias y los edificios escolares. (Suponiendo que el cálculo entre distancias con la fórmula de Haservine está correcto).

Voy a suponer que hay distancias muy grandes entre ambos puntos por que pudo haber alguno de los siguientes errores.
- Error al capturar coordenadas.
- Mala asignación de códigos postales.
- Ubicación de colonias / escuelas inadecuada.
- Edificios en zonas rurales donde el CP cubre áreas extensas.
- No todas las ubicaciones son escuelas, también hay oficinas.
